In [45]:
import time
import json
import requests
import pymysql
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine

## 上傳資料庫

In [46]:
def uploadDB(df, table_name):
    database_username = 'jackyfu'
    database_password = 'data8756'
    database_ip       = 'sg2nlmysql29plsk.secureserver.net'
    database_name     = 'Bigdata'
    sql_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(database_username, database_password, database_ip, database_name))
    df.astype(str).to_sql(name=table_name, con=sql_engine, if_exists = 'append', index=False)
    print('Successfully insert into Bigdata table: ' + table_name)

## 爬取DCARD版位資訊 API 網址：https://www.dcard.tw/service/api/v2

### 全部文章	GET	/posts
### 看板資訊	GET	/forums
### 看板內文章列表	GET	/forums/{看板名稱}/posts
### 文章內文	GET	/posts/{文章ID}
### 文章內引用連結	GET	/posts/{文章ID}/links
### 文章內留言	GET	/posts/{文章ID}/comments

In [47]:
def get_all_forums_and_upload():
    url_path = 'https://www.dcard.tw/service/api/v2/forums'
    df = get_df_from_dcard_api(url_path)
    df = df[['id', 'alias', 'name', 'subscriptionCount', 'postCount', 'isSchool', 'createdAt']]
    df.postCount = df.postCount.apply(lambda x : x['last30Days'])
    df.createdAt = df.createdAt.apply(lambda x : x[:10])
    uploadDB(df, 'dcard_forums')
    return df

In [48]:
def get_df_from_dcard_api(url):
    response = requests.get(url).text
    data = json.loads(response)
    df = pd.DataFrame(data)
    return df

In [49]:
def get_latest_post_from_forums(forums_name, post_count):
    postid = ''
    max_limit = 100
    iteration = int(post_count/max_limit)
    df_post = pd.DataFrame()
    for i in range(0,iteration):
        post_url = 'https://www.dcard.tw/service/api/v2/forums/' + forums_name + '/posts?popular=false&limit=' + str(max_limit)
        suffix = '&before=' + str(postid)
        if postid != '':   
            post_url = post_url + suffix
        df_post_batch = get_df_from_dcard_api(post_url)
        df_post = df_post.append(df_post_batch)
        postid = df_post_batch.tail(1).id.tolist()[0]
        time.sleep(3)
    return df_post

In [50]:
# 建立版位資訊表
df_forums = get_all_forums_and_upload()

Successfully insert into Bigdata table: dcard_forums


In [51]:
# 頻繁的呼叫API會被擋
# 擷取最熱門版位，最近1000篇發文資訊，並上傳
forums_list = df_forums.sort_values(by = 'subscriptionCount', ascending = False).head(10).alias.tolist()
for forums_name in forums_list:
    time.sleep(10)
    print(forums_name)
    df_post = get_latest_post_from_forums(forums_name, 1000)
    df_post = df_post[['id', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'school', 'gender', 'createdAt']]
    df_post.columns = ['postid', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'author', 'gender', 'createdAt']
    uploadDB(df_post, 'dcard_posts')

sex


JSONDecodeError: Expecting value: line 1 column 1 (char 0)